In [34]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Input, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint
import keras
from sklearn.metrics import roc_auc_score
import numpy as np
from IPython.core.debugger import Tracer
from datetime import datetime

In [35]:
top_model_weights_path = 'bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'Data/train'
validation_data_dir = 'Data/val'
nb_train_samples = 1800
nb_validation_samples = 200
epochs = 10 #50
batch_size = 25

In [36]:
class auc_roc_callback(keras.callbacks.Callback):
    def __init__(self, val_data_generator, val_labels, weight_file_path):
        self.val_data_generator = val_data_generator
        self.val_labels = val_labels
        self.val_samples = val_labels.shape[0]
        self.weight_file_path = weight_file_path
        self.best_AUC_Score = float("-inf")
    
    def on_train_begin(self, logs={}):
        self.auc_history = []
        self.loss = []
 
    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict_generator(self.val_data_generator, self.val_samples)
        self.auc_history.append(roc_auc_score(self.val_labels, y_pred))
        print '\n AUC Score: ', self.auc_history[-1]
        
        #Saving the model weights if the AUC score is the best observed till now
        if self.best_AUC_Score < self.auc_history[-1]:
            dateTag = str(datetime.now().replace(second=0, microsecond=0)).replace(' ', '_').replace('-', '_').replace(':', '_')
            filepath = self.weight_file_path.format(str(round(self.auc_history[-1] * 100, 5)).replace('.', '_'), dateTag)
            print('Epoch %05d: AUC improved from %0.5f to %0.5f,'
                    ' saving model to %s'
                    % (epoch + 1, self.best_AUC_Score, self.auc_history[-1],
                       filepath))
            self.best_AUC_Score = self.auc_history[-1]
            self.model.save_weights(filepath, overwrite=True)
        return
 
    def on_batch_begin(self, batch, logs={}):
        return
 
    def on_batch_end(self, batch, logs={}):
        self.loss.append(logs.get('loss'))
        return

In [37]:
input = Input(shape=(3, img_width, img_height),name = 'image_input')
model = applications.VGG16(weights='imagenet', include_top=False, input_tensor = input)
print('Model loaded.', model.output_shape[1:])

('Model loaded.', (512, 4, 4))


In [38]:
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

In [39]:
train_data = np.load(open('bottleneck_features_train.npy'))
train_data.shape

(1800, 512, 4, 4)

In [40]:
train_x = np.random.random_sample((24, 3,150,150))
train_y = (np.random.random_sample(24) > 0.8).astype(int)

val_x = np.random.random_sample((24, 3,150,150))
val_y = (np.random.random_sample(24) > 0.8).astype(int)

In [43]:
# add the model on top of the convolutional base
#Freezing the top 15 layers i.e. just before the last conv layer
for layer in model.layers[:15]:
    layer.trainable = False
    
mdl = Model(input= model.input, output= top_model(model.output))

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)


# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
mdl.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9, nesterov = True, decay = 0.05),
              metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255)
'''
    rotation_range=95,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    width_shift_range = 0.2,
    height_shift_range = 0.2)
'''
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False)


#Creating the callbacks
val_datagen = ImageDataGenerator(rescale=1. / 255)
val_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False)

val_labels = np.array([0]*167 + [1]*33)
#checkpointer = ModelCheckpoint(filepath="trainedmodel/weights_fineTuning_layersFreezed_AUC_{}_time_{}.hdf5", verbose=1, save_best_only=True)
auc_roc_hist = auc_roc_callback(val_generator, val_labels, "trainedmodel/weights_fineTuning_layersFreezed_AUC_{}_time_{}.hdf5")

# fine-tune the model
class_weight = {0 : 1.,
    1: 6.}
mdl.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples // batch_size,
    nb_epoch=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples,
    callbacks=[auc_roc_hist],
    class_weight = class_weight)
'''

mdl.fit(
    train_x,
    train_y,
    #samples_per_epoch=nb_train_samples,
    batch_size = batch_size,
    nb_epoch=epochs,
    validation_data=(val_x, val_y),
    #nb_val_samples=nb_validation_samples,
    callbacks=[auc_roc_hist, checkpointer])
'''

Found 1800 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Epoch 1/10
50/72 [===================>..........] - ETA: 6s - loss: 0.3262 - acc: 0.8800 
 AUC Score:  0.599165305752
Epoch 00001: AUC improved from -inf to 0.59917, saving model to trainedmodel/weights_fineTuning_layersFreezed_AUC_59_91653_time_2017_10_29_18_49_00.hdf5
75/72 [===============================] - 302s - loss: 0.7711 - acc: 0.8667 - val_loss: 0.7683 - val_acc: 0.5800
Epoch 2/10
50/72 [===================>..........] - ETA: 4s - loss: 1.1147 - acc: 0.7000
 AUC Score:  0.417347123934
75/72 [===============================] - 280s - loss: 1.2958 - acc: 0.6533 - val_loss: 1.2839 - val_acc: 0.2250
Epoch 3/10
50/72 [===================>..........] - ETA: 4s - loss: 1.2673 - acc: 0.3000
 AUC Score:  0.546361821811
75/72 [===============================] - 287s - loss: 1.2266 - acc: 0.3200 - val_loss: 1.4507 - val_acc: 0.1850
Epoch 4/10
50/72 [==============

'\n\nmdl.fit(\n    train_x,\n    train_y,\n    #samples_per_epoch=nb_train_samples,\n    batch_size = batch_size,\n    nb_epoch=epochs,\n    validation_data=(val_x, val_y),\n    #nb_val_samples=nb_validation_samples,\n    callbacks=[auc_roc_hist, checkpointer])\n'

## Trying with adaptive learning rate

In [25]:
input = Input(shape=(3, img_width, img_height),name = 'image_input')
model_adaptive = applications.VGG16(weights='imagenet', include_top=False, input_tensor = input)
print('Model loaded.', model_adaptive.output_shape[1:])

# add the model on top of the convolutional base
#Freezing the top 15 layers i.e. just before the last conv layer
for layer in model_adaptive.layers[:15]:
    layer.trainable = False
    
mdl_adap = Model(input= model_adaptive.input, output= top_model(model_adaptive.output))

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)


# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
mdl_adap.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(rescale=1. / 255)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False)


#Creating the callbacks
val_datagen = ImageDataGenerator(rescale=1. / 255)
val_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False)

val_labels = np.array([0]*167 + [1]*33)
#checkpointer = ModelCheckpoint(filepath="trainedmodel/weights_fineTuning_layersFreezed_AUC_{}_time_{}.hdf5", verbose=1, save_best_only=True)
auc_roc_hist_adap = auc_roc_callback(val_generator, val_labels, "trainedmodel/weights_fineTuning_AdamOpt_layersFreezed_AUC_{}_time_{}.hdf5")

# fine-tune the model

mdl_adap.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples // batch_size,
    nb_epoch=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples,
    callbacks=[auc_roc_hist_adap])

('Model loaded.', (512, 4, 4))
Found 1800 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


INFO (theano.gof.compilelock): Refreshing lock /Users/shashankbhushan/.theano/compiledir_Darwin-16.7.0-x86_64-i386-64bit-i386-2.7.14-64/lock_dir/lock


Epoch 1/10
50/72 [===================>..........] - ETA: 6s - loss: 1.3909 - acc: 0.8800 
 AUC Score:  0.5
Epoch 00001: AUC improved from -inf to 0.50000, saving model to trainedmodel/weights_fineTuning_AdamOpt_layersFreezed_AUC_50_0_time_2017_10_29_16_52_00.hdf5
75/72 [===============================] - 288s - loss: 1.1421 - acc: 0.9067 - val_loss: 2.6595 - val_acc: 0.8350
Epoch 2/10
50/72 [===================>..........] - ETA: 4s - loss: 4.1907 - acc: 0.7400
 AUC Score:  0.5
75/72 [===============================] - 279s - loss: 3.2236 - acc: 0.8000 - val_loss: 2.6595 - val_acc: 0.8350
Epoch 3/10
50/72 [===================>..........] - ETA: 4s - loss: 2.9013 - acc: 0.8200
 AUC Score:  0.5
75/72 [===============================] - 277s - loss: 2.5789 - acc: 0.8400 - val_loss: 2.6595 - val_acc: 0.8350
Epoch 4/10
50/72 [===================>..........] - ETA: 4s - loss: 3.2236 - acc: 0.8000
 AUC Score:  0.5
75/72 [===============================] - 279s - loss: 3.2236 - acc: 0.8000 - v

KeyboardInterrupt: 

In [20]:
auc_roc_hist.auc_history

[0.6875,
 0.59999999999999998,
 0.70000000000000007,
 0.59999999999999998,
 0.71250000000000002,
 0.59999999999999998,
 0.71250000000000002,
 0.59999999999999998,
 0.71250000000000002,
 0.59999999999999998]

In [38]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
image_input (InputLayer)         (None, 3, 150, 150)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 64, 150, 150)  1792        image_input[0][0]                
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 64, 150, 150)  36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 64, 75, 75)    0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [40]:
for i, layer in enumerate(model.layers[:25]):
    print i, layer

0 <keras.engine.topology.InputLayer object at 0x1c279e0250>
1 <keras.layers.convolutional.Convolution2D object at 0x1c279e0110>
2 <keras.layers.convolutional.Convolution2D object at 0x1c1effd390>
3 <keras.layers.pooling.MaxPooling2D object at 0x1138228d0>
4 <keras.layers.convolutional.Convolution2D object at 0x113822a90>
5 <keras.layers.convolutional.Convolution2D object at 0x11377d650>
6 <keras.layers.pooling.MaxPooling2D object at 0x113783f90>
7 <keras.layers.convolutional.Convolution2D object at 0x113783b90>
8 <keras.layers.convolutional.Convolution2D object at 0x11384d450>
9 <keras.layers.convolutional.Convolution2D object at 0x113853bd0>
10 <keras.layers.pooling.MaxPooling2D object at 0x1c1ce13710>
11 <keras.layers.convolutional.Convolution2D object at 0x1c1ce138d0>
12 <keras.layers.convolutional.Convolution2D object at 0x1c1ce36490>
13 <keras.layers.convolutional.Convolution2D object at 0x1c1ce4aed0>
14 <keras.layers.pooling.MaxPooling2D object at 0x1c1ce67a90>
15 <keras.layers.c

In [42]:
for i, layer in enumerate(model.layers[:15]):
    print i, layer

0 <keras.engine.topology.InputLayer object at 0x1c279e0250>
1 <keras.layers.convolutional.Convolution2D object at 0x1c279e0110>
2 <keras.layers.convolutional.Convolution2D object at 0x1c1effd390>
3 <keras.layers.pooling.MaxPooling2D object at 0x1138228d0>
4 <keras.layers.convolutional.Convolution2D object at 0x113822a90>
5 <keras.layers.convolutional.Convolution2D object at 0x11377d650>
6 <keras.layers.pooling.MaxPooling2D object at 0x113783f90>
7 <keras.layers.convolutional.Convolution2D object at 0x113783b90>
8 <keras.layers.convolutional.Convolution2D object at 0x11384d450>
9 <keras.layers.convolutional.Convolution2D object at 0x113853bd0>
10 <keras.layers.pooling.MaxPooling2D object at 0x1c1ce13710>
11 <keras.layers.convolutional.Convolution2D object at 0x1c1ce138d0>
12 <keras.layers.convolutional.Convolution2D object at 0x1c1ce36490>
13 <keras.layers.convolutional.Convolution2D object at 0x1c1ce4aed0>
14 <keras.layers.pooling.MaxPooling2D object at 0x1c1ce67a90>


In [24]:
mdl.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
image_input (InputLayer)         (None, 3, 150, 150)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 64, 150, 150)  0           image_input[0][0]                
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 64, 150, 150)  0           block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 64, 75, 75)    0           block1_conv2[0][0]               
___________________________________________________________________________________________

INFO (theano.gof.compilelock): Refreshing lock /Users/shashankbhushan/.theano/compiledir_Darwin-16.7.0-x86_64-i386-64bit-i386-2.7.14-64/lock_dir/lock


____________________________________________________________________________________________________
block5_conv3 (Convolution2D)     (None, 512, 9, 9)     2359808     block5_conv2[0][0]               
____________________________________________________________________________________________________
block5_pool (MaxPooling2D)       (None, 512, 4, 4)     0           block5_conv3[0][0]               
____________________________________________________________________________________________________
sequential_3 (Sequential)        (None, 1)             2097665     block5_pool[0][0]                
Total params: 9177089
____________________________________________________________________________________________________
